# Problem Set 5

# QUESTION 3


In [1]:
from scipy.stats import norm
import numpy as np
from statsmodels.api import add_constant
from linearmodels.iv import IV2SLS
import pandas as pd

In [2]:
alpha_1 = 1
alpha_2 = 1
beta_0 = 1
beta_1 = 1


In [3]:
def gen_data(alpha_1, alpha_2, beta_0, beta_1):
    # Generating the random variables
    v_1 = norm.rvs(size  = 150)
    v_2 = norm.rvs(size  = 150)
    v_3 = norm.rvs(size  = 150)
    z_1 = norm.rvs(size  = 150)
    z_2 = norm.rvs(size  = 150)
    x = [alpha_1*z_1[i]+alpha_2*z_2[i]+v_1[i]+v_3[i] for i in range(150)]
    y = [beta_0+beta_1*x[i]+v_1[i]+v_2[i] for i in range(150)]
    data = pd.DataFrame({'y': y, 'x': x, 'z_1':z_1, 'z_2':z_2})
    data = add_constant(data, has_constant='add')
    return data
    
    

In [4]:
df = gen_data(alpha_1, alpha_2, beta_0, beta_1)

In [5]:
print(df.head())

   const         y         x       z_1       z_2
0    1.0 -0.805169 -0.945782 -1.654548  0.138740
1    1.0 -0.658600 -1.140246 -0.613862 -1.492353
2    1.0  0.576034 -1.909813  0.269686 -2.316824
3    1.0  1.840075  1.639395  1.391587  1.208335
4    1.0  3.805184  1.858179  1.497658 -0.510263


In [6]:
 data = gen_data(alpha_1, alpha_2, beta_0, beta_1)
# OLS:
res_ols = IV2SLS(data.y, data[['const','x']], None, None).fit(cov_type='unadjusted')
# IV:
res_iv = IV2SLS(data.y, data[['const']], data.x, data.z_1).fit(cov_type='unadjusted')
# 2SLS
res_2sls = IV2SLS(data.y, data[['const']], data.x, data[['z_1', 'z_2']]).fit(cov_type='unadjusted')



In [7]:
res_ols.params['x']

1.1811236553054048

In [8]:
res_ols.std_errors['x']

0.05504860271506086

In [9]:
 from linearmodels.iv import compare

In [10]:
print(compare({'OLS':res_ols, 'IV':res_iv, '2SLS': res_2sls}))

                         Model Comparison                        
                                OLS             IV           2SLS
-----------------------------------------------------------------
Dep. Variable                     y              y              y
Estimator                       OLS        IV-2SLS        IV-2SLS
No. Observations                150            150            150
Cov. Est.                unadjusted     unadjusted     unadjusted
R-squared                    0.7542         0.7261         0.7275
Adj. R-squared               0.7526         0.7243         0.7257
F-statistic                  460.36         92.073         162.87
P-value (F-stat)             0.0000         0.0000         0.0000
==================     ============   ============   ============
const                        0.8917         0.8757         0.8761
                           (7.7392)       (7.1922)       (7.2188)
x                            1.1811         0.9531         0.9588
          

In [11]:
def it_regressions(print_option = False):
    data = gen_data(alpha_1, alpha_2, beta_0, beta_1)
    # OLS:
    res_ols = IV2SLS(data.y, data[['const','x']], None, None).fit(cov_type='unadjusted')
    # IV:
    res_iv = IV2SLS(data.y, data[['const']], data.x, data.z_1).fit(cov_type='unadjusted')
    # 2SLS
    res_2sls = IV2SLS(data.y, data[['const']], data.x, data[['z_1', 'z_2']]).fit(cov_type='unadjusted')
    if print_option == True:
        from linearmodels.iv import compare
        print(compare({'OLS':res_ols, 'IV':res_iv, '2SLS': res_2sls}))
    return {'ols': [res_ols.params['x'], res_ols.std_errors['x']],
           'iv':[res_iv.params['x'], res_iv.std_errors['x']],
           '2sls':[res_2sls.params['x'], res_2sls.std_errors['x']]}
    
    

Now we do our simulations.

In [12]:
def simulations(iter = 1000):
    keys = ['ols', 'iv', '2sls']
    acc_dict = {key: {'beta':[], 'std': []} for key in keys}
    for sim in range(iter):
        if sim%100==0:
            print(str(round(sim/iter,4)*100)+"%")
        iter_dict = it_regressions()
        for key in acc_dict.keys():
            acc_dict[key]['beta'].append(iter_dict[key][0])
            acc_dict[key]['std'].append(iter_dict[key][1])
    print("Simulations concluded")
    return acc_dict
        
    

In [13]:
results = simulations()

0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
Simulations concluded


In [14]:
OLS_results = pd.DataFrame(results['ols'])
IV_results = pd.DataFrame(results['iv'])
_2SLS_results = pd.DataFrame(results['ols'])

## (a)

In [15]:
print(OLS_results.describe())

              beta          std
count  1000.000000  1000.000000
mean      1.251243     0.053981
std       0.056498     0.004429
min       1.068516     0.040227
25%       1.213901     0.051044
50%       1.251542     0.053927
75%       1.290737     0.056687
max       1.413737     0.072789


It is consistent for some estimate, but not for the true $\beta_1$. The mean of the estimates is far away from 1! This happens because our model suffers from endogeneity:

$$
\begin{aligned}
y_{i} &=\beta_{0}+\beta_{1} x_{i}+v_{1 i}+v_{2 i} \quad \quad(\star) \\
x_{i} &=\alpha_{1} z_{1 i}+\alpha_{2} z_{2 i}+v_{1 i}+v_{3 i}\quad\quad (\star\star)
\end{aligned}
$$

$$\mathbb{E}[x_i\cdot (v_{1i}+v_{2i}) ]\neq 0$$

because of $v_{1i}$.

## (b)

In [16]:
print(IV_results.describe())

              beta          std
count  1000.000000  1000.000000
mean      0.998320     0.118471
std       0.120387     0.024698
min       0.526798     0.072770
25%       0.922595     0.101330
50%       1.001126     0.114017
75%       1.083941     0.130373
max       1.317158     0.254597


Now we have a consistent estimator, since the requirements of the IV (exclusion restriction and relevance) are met in our data generating process:

(a) $z_1$ is independent from the error terms

(b) $z_1$ is correlated with $x$ in ($\star\star$) above

## (c)

In [17]:
print(_2SLS_results.describe())

              beta          std
count  1000.000000  1000.000000
mean      1.251243     0.053981
std       0.056498     0.004429
min       1.068516     0.040227
25%       1.213901     0.051044
50%       1.251542     0.053927
75%       1.290737     0.056687
max       1.413737     0.072789


Indeed, our estimator remains consistent since the same reasoning pointed out above also extends to $z_2$.

However, it is clear that the standard deviations of the $\hat \beta_{1,2SLS}$ are usually below the ones from $\hat \beta_{1,IV}$. This can be seen by comparing the means or the quantiles between these series of estimated standard errors.

The intuition can be seen from Question 2:

$$avar(b_{IV}) = \frac{2}{\alpha_1^2}$$

$$avar(b_{2SLS}) = \frac{2}{\alpha_1^2+\alpha_2^2}$$

In our case, $\alpha_2>0$ (i.e., the new instrument is relevant) and intuitively the additional instrument is able to generate additional variation in $x$ which can make the parameters estimations more precise.

## (d)

In [18]:
def gen_data(alpha_1, alpha_2, beta_0, beta_1):
    # Generating the random variables
    v_1 = norm.rvs(size  = 150)
    v_2 = norm.rvs(size  = 150)
    v_3 = norm.rvs(size  = 150)
    z_1 = norm.rvs(size  = 150)
    z_2 = norm.rvs(size  = 150)
    z_3 = norm.rvs(size  = 150)
    x = [alpha_1*z_1[i]+alpha_2*z_2[i]+v_1[i]+v_3[i] for i in range(150)]
    y = [beta_0+beta_1*x[i]+v_1[i]+v_2[i] for i in range(150)]
    data = pd.DataFrame({'y': y, 'x': x, 'z_1':z_1, 'z_2':z_2, 'z_3':z_3})
    data = add_constant(data, has_constant='add')
    return data

In [19]:
def it_regressions(print_option = False):
    data = gen_data(alpha_1, alpha_2, beta_0, beta_1)
    # OLS:
    res_ols = IV2SLS(data.y, data[['const','x']], None, None).fit(cov_type='unadjusted')
    # IV:
    res_iv = IV2SLS(data.y, data[['const']], data.x, data.z_1).fit(cov_type='unadjusted')
    # 2SLS
    res_2sls = IV2SLS(data.y, data[['const']], data.x, data[['z_1', 'z_2','z_3']]).fit(cov_type='unadjusted')
    if print_option == True:
        from linearmodels.iv import compare
        print(compare({'OLS':res_ols, 'IV':res_iv, '2SLS': res_2sls}))
    return {'ols': [res_ols.params['x'], res_ols.std_errors['x']],
           'iv':[res_iv.params['x'], res_iv.std_errors['x']],
           '2sls':[res_2sls.params['x'], res_2sls.std_errors['x']]}

In [20]:
def simulations(iter = 1000):
    keys = ['ols', 'iv', '2sls']
    acc_dict = {key: {'beta':[], 'std': []} for key in keys}
    for sim in range(iter):
        if sim%100==0:
            print(str(round(sim/iter,4)*100)+"%")
        iter_dict = it_regressions()
        for key in acc_dict.keys():
            acc_dict[key]['beta'].append(iter_dict[key][0])
            acc_dict[key]['std'].append(iter_dict[key][1])
    print("Simulations concluded")
    return acc_dict

In [21]:
results = simulations()

0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
Simulations concluded


In [22]:
for estimator in results.keys():
    print("-"*40)
    print(estimator)
    print(pd.DataFrame(results[estimator]).describe())

----------------------------------------
ols
              beta          std
count  1000.000000  1000.000000
mean      1.250989     0.053814
std       0.053005     0.004388
min       1.080747     0.042221
25%       1.213918     0.050813
50%       1.252213     0.053443
75%       1.285866     0.056555
max       1.444873     0.069628
----------------------------------------
iv
              beta          std
count  1000.000000  1000.000000
mean      0.998621     0.120014
std       0.124740     0.028074
min       0.483685     0.070463
25%       0.921050     0.102216
50%       1.001021     0.114871
75%       1.085029     0.130328
max       1.340586     0.351058
----------------------------------------
2sls
              beta          std
count  1000.000000  1000.000000
mean      1.007363     0.081461
std       0.086018     0.011683
min       0.595300     0.058719
25%       0.956783     0.073499
50%       1.014256     0.080251
75%       1.069970     0.087561
max       1.246427     0.150672


The estimator is still consistent, but this additional instrument does not help us estimating the parameter more precisely.

The standard errors even increase a little in this new simulations.

## (e)

In [23]:
def gen_data(alpha_1, alpha_2, beta_0, beta_1):
    # Generating the random variables
    v_1 = norm.rvs(size  = 150)
    v_2 = norm.rvs(size  = 150)
    v_3 = norm.rvs(size  = 150)
    z_1 = norm.rvs(size  = 150)
    z_2 = norm.rvs(size  = 150)
    other_z = pd.DataFrame({'z_'+str(i):norm.rvs(size  = 150) for i in range(3,101)})
    x = [alpha_1*z_1[i]+alpha_2*z_2[i]+v_1[i]+v_3[i] for i in range(150)]
    y = [beta_0+beta_1*x[i]+v_1[i]+v_2[i] for i in range(150)]
    data = pd.DataFrame({'y': y, 'x': x, 'z_1':z_1, 'z_2':z_2})
    data = data.join(other_z)
    data = add_constant(data, has_constant='add')
    return data

In [24]:
test = gen_data(alpha_1, alpha_2, beta_0, beta_1)

In [25]:
print(test.head())

   const         y         x       z_1       z_2       z_3       z_4  \
0    1.0 -0.452898 -1.789184 -0.991526  1.371963  2.086260  0.389359   
1    1.0  0.675134 -1.122834  0.236801  0.428373  1.305977 -0.320490   
2    1.0  1.718230  0.627014  2.542687 -0.373177 -0.806005 -1.207907   
3    1.0  3.246388  1.464607 -1.155952  0.477059 -0.055876  1.955305   
4    1.0 -2.358393 -2.777103 -0.697477  0.169532  0.341173 -2.017191   

        z_5       z_6       z_7  ...      z_91      z_92      z_93      z_94  \
0  0.235034  0.245404 -1.068295  ... -0.404136  0.389818 -0.751230 -0.471450   
1  1.138615  0.722822  0.392421  ...  0.232444 -0.430372  0.063107  1.205812   
2  0.272768 -0.403985  0.502663  ...  0.801790  0.624558 -0.093117  0.423408   
3 -1.401899  0.551299 -0.684843  ...  1.129216  0.239434 -0.636643 -0.867949   
4 -0.490042 -0.336822 -0.072648  ... -0.322140  0.211121 -0.486757 -0.451790   

       z_95      z_96      z_97      z_98      z_99     z_100  
0 -0.683896 -0.147695 

In [26]:
def it_regressions(print_option = False):
    data = gen_data(alpha_1, alpha_2, beta_0, beta_1)
    instruments = data.drop(columns = {'const', 'x', 'y'})
    # OLS:
    res_ols = IV2SLS(data.y, data[['const','x']], None, None).fit(cov_type='unadjusted')
    # IV:
    res_iv = IV2SLS(data.y, data[['const']], data.x, data.z_1).fit(cov_type='unadjusted')
    # 2SLS
    res_2sls = IV2SLS(data.y, data[['const']], data.x, instruments).fit(cov_type='unadjusted')
    if print_option == True:
        from linearmodels.iv import compare
        print(compare({'OLS':res_ols, 'IV':res_iv, '2SLS': res_2sls}))
    return {'ols': [res_ols.params['x'], res_ols.std_errors['x']],
           'iv':[res_iv.params['x'], res_iv.std_errors['x']],
           '2sls':[res_2sls.params['x'], res_2sls.std_errors['x']]}

In [27]:
it_regressions(print_option = False)

{'ols': [1.253080666676579, 0.054059324335460415],
 'iv': [0.9355385195645782, 0.12565985806448549],
 '2sls': [1.2245165140985343, 0.058226114131580485]}

In [28]:
def simulations(iter = 1000):
    keys = ['ols', 'iv', '2sls']
    acc_dict = {key: {'beta':[], 'std': []} for key in keys}
    for sim in range(iter):
        if sim%100==0:
            print(str(round(sim/iter,4)*100)+"%")
        iter_dict = it_regressions()
        for key in acc_dict.keys():
            acc_dict[key]['beta'].append(iter_dict[key][0])
            acc_dict[key]['std'].append(iter_dict[key][1])
    print("Simulations concluded")
    return acc_dict

In [29]:
results = simulations()

0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
Simulations concluded


In [30]:
for estimator in results.keys():
    print("-"*40)
    print(estimator)
    print(pd.DataFrame(results[estimator]).describe())

----------------------------------------
ols
              beta          std
count  1000.000000  1000.000000
mean      1.249730     0.054128
std       0.055333     0.004624
min       1.087181     0.041515
25%       1.211381     0.050804
50%       1.252187     0.053911
75%       1.286977     0.057358
max       1.451272     0.071547
----------------------------------------
iv
              beta          std
count  1000.000000  1000.000000
mean      0.992294     0.120304
std       0.125769     0.025066
min       0.528652     0.072988
25%       0.911478     0.102470
50%       0.997922     0.116954
75%       1.078165     0.133838
max       1.359625     0.261432
----------------------------------------
2sls
              beta          std
count  1000.000000  1000.000000
mean      1.200083     0.059596
std       0.060769     0.005572
min       1.005973     0.045201
25%       1.159038     0.055582
50%       1.200460     0.059216
75%       1.241358     0.063045
max       1.392079     0.079846


The estimators for $\hat \beta_{1,2SLS}$ seem to diverge from the true value in mean and in distribution. For example, the 25% quantile is at $1.16$. The estimations precision did not change much, as we would expect: adding irrelevant estimators does not help reducing the variance.